# RAG Використання Azure AI Agent Service та Semantic Kernel

Цей фрагмент коду демонструє, як створити та керувати агентом Azure AI для генерації з підкріпленням контекстом (RAG) за допомогою `Azure AI Agent Service` та `Semantic Kernel`. Агент обробляє запити користувачів на основі отриманого контексту та відповідно надає точні відповіді.


Виправлення версії SQLite  
Якщо ви стикаєтеся з помилкою:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Розкоментуйте цей блок коду на початку вашого нотатника:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Імпорт пакетів
Наступний код імпортує необхідні пакети:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Генерація з доповненням пошуку за допомогою Semantic Kernel та Azure AI Agent Service

Цей приклад демонструє, як використовувати **Azure AI Agent Service** для виконання **генерації з доповненням пошуку (RAG)**, поєднуючи мовну модель із контекстом, специфічним для домену, з завантаженого документа.

### Як це працює

1. **Завантаження документа**: Файл у форматі markdown (document.md), що містить інформацію (страхова політика Contoso для подорожей), завантажується до сервісу агента.

2. **Створення векторного сховища**: Документ індексується у векторному сховищі для забезпечення семантичного пошуку його змісту.

3. **Конфігурація агента**: Агент створюється з використанням моделі `gpt-4o` із наступними строгими інструкціями:
   - Відповідати на запитання лише на основі отриманого змісту з документа.
   - Відмовлятися відповідати, якщо запитання виходить за межі теми.

4. **Інтеграція інструменту пошуку файлів**: `FileSearchTool` реєструється з агентом, що дозволяє моделі шукати та отримувати відповідні фрагменти з індексованого документа під час виконання запитів.

5. **Взаємодія з користувачем**: Користувачі можуть ставити запитання. Якщо у документі знайдено відповідну інформацію, агент генерує обґрунтовану відповідь.  
   Якщо ні, агент чітко відповідає, що документ не містить достатньої інформації.


### Головна функція


Переконайтеся, що спочатку виконали команду `az login` за допомогою Azure CLI, щоб забезпечити належний контекст автентифікації під час використання `DefaultAzureCredential`. Служба Azure AI Agent не використовує ключі API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Звісно! Будь ласка, надайте текст файлу Markdown, який потрібно перекласти.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
